## Ejercicios propuestos: *Threading*

Los siguientes problemas se dejan como opción para ejercitar los conceptos revisados en el material sobre *theading* (semana-05). Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una issue en el foro del curso.

El objetivo es poner en práctica el uso de *threads* y como esto afecta el flujo de un programa. A su vez, se busca vean en concreto los problemas de sincronización y comunicación entre *threads*. Este material nos acerca mucho a entender como funciona el sistema operativo de nuestros computadores, y nos permite modelar más situaciones.

In [ ]:
import threading
import time

## Problema 1

A continuación, se te presenta la función `cuenta_hasta_diez`. Esta, como sugiere el nombre, cuenta hasta a diez desde el número uno, pero el tiempo (en segundos) que demora entre cada número es aleatorio entre uno y cinco segundos. Usando esta función, instancia 5 *threads* distintos, con nombres distintivos, y ejecútalos para que ejecuten la función simultáneamente. Una vez que escribas el código necesario, ejecútalo varias veces y ve el resultado que se produce.

In [ ]:
from random import randint

def cuenta_hasta_diez():
    nombre_thread = threading.current_thread().name
    for numero in range(1, 11):
        time.sleep(randint(1, 5))
        print(f"{nombre_thread}: {numero}...")
        
# Instancia 5 threads distintos y ejecútalos.



## Problema 2

Ahora, de forma similar al problema anterior, se te presenta la función `deletrea_palabra`. Esta recibe como argumentos un *string* en `palabra` y un `int` en `periodo`. Esta imprime en orden las letras del *string* (`palabra`) entregado cada cierta cantidad de segundos, definido por `periodo`. 

Debes instanciar tres *threads* distintos que ejecuten esta función y ejecutalos simultaneamente, con los siguientes parámetros:
- `"flZ1"` y `3`
- `"e  cIUes"` y `5`
- `"i8Hq"` y `7`

In [ ]:
def deletrea_palabra(palabra, periodo):
    for caracter in palabra:
        time.sleep(periodo)
        print(caracter.upper())


# Instancia 3 threads distintos y ejecútalos con los parametros dados.



## Problema 3

Tienes tres funciones de cuenta, las cuales funcionan a diferentes ritmos, o sea, cuentan en diferentes cantidades.
 - `cuenta_1` cuenta de a 1 número cada 3 seg.
 - `cuenta_2` cuenta de a 2 números cada 2 seg.
 - `cuenta_3` cuenta de a 3 números cada 1 seg.
 
Además, `cuenta_1` inicia un *thread* que ejecuta a `cuenta_2`, que a su vez inicia a `cuenta_3`. Esto produce que los tres conteos se inicien (casi) simultaneamente.
 
Tú misión es editar el código de forma que `cuenta_2` solo comienza a contar una vez que `cuenta_3` terminó de contar; de la misma forma, que `cuenta_1` solo comience a contar una vez que `cuenta_2` terminó de contar; Y finalmente que el mensaje `"¡Todos terminaron en orden!"` que se imprime en el programa principal solo lo haga una vez que todos los contadores terminaron de contar. Es decir, deberías esperar que primero el contador de a 3 cuente, luego el contador de 2, luego el contador de 1 y finalmente se imprimea el mensaje `"¡Todos terminaron en orden!"`.

Usa el método `join` de `Thread` para lograr el objetivo, y **solo debes agregar líneas al código presentado, no eliminar ni modificar líneas existentes**.

In [ ]:
def cuenta_3():
    for i in range(1, 40, 3):
        print(f"Contando de a 3: {i}")
        time.sleep(1)
    print("-- ¡Terminó cuenta 3! --")

def cuenta_2():
    thread = threading.Thread(target=cuenta_3)
    thread.start()
    for i in range(1, 9, 2):
        print(f"Contando de a 2: {i}")
        time.sleep(2) 
    print("-- ¡Terminó cuenta 2! --")

def cuenta_1():
    thread = threading.Thread(target=cuenta_2)
    thread.start()
    for i in range(1, 9):
        print(f"Contando de a 1: {i}")
        time.sleep(3)
    print("-- ¡Terminó cuenta 1! --")


    
thread = threading.Thread(target=cuenta_1)
thread.start()
print("¡Todos terminaron en orden!")

## Problema 4

A continuación se te presenta la simulación de un restoran. Clientes pueden hacer pedidos de productos alimenticios y estos están almacenados en una cola. Para atender a los clientes, se ponen a trabajar a tres *threads* que ejecutan la misma misma función `atender_pedidos`. Pero estos trabajadores son medios torpes, cuando intentan generar un pedido, tiene una probabilidad de 50% de que se les caiga y no se complete el pedido. Además, los tres no se coordinan y suelen trabajar en el mismo pedido al mismo tiempo.

Tu objetivo es arreglar esta situación para que logren coordinarse en la elaboración de pedidos. Utiliza un `Lock` para arreglarlo, y **solo debes agregar líneas al código presentado, no eliminar ni modificar líneas existentes**.

In [ ]:
from collections import deque
from random import randint

clientes = {
    "Enzo": [],
    "Dani": [],
    "Dante": [],
    "Josefina": [],
    "Ian": []
}
cola_de_pedidos = deque([("Enzo", "🍕"), ("Josefina", "🍣"), ("Dante", "🌭"), ("Dani", "🍟"), ("Ian", "🍔"), ("Ian", "🍰"), ("Enzo", "🌮"), ("Dani", "🍩"), ("Enzo", "🍫")])


def atender_pedidos(cola, clientes):
    while len(cola) > 0:
        print("¡Haré un pedido!")
        cliente, comida = cola[0]
        print(f"Preparando {comida} para {cliente}")
        time.sleep(1)
        if randint(0, 1) == 1:
            bandeja = clientes[cliente]
            bandeja.append(comida)
            print(f"Pedido de {comida} entregado a {cliente}")
            cola.popleft()
        else:
            print("¡Se me cayó!")
        

trabajador_1 = threading.Thread(target=atender_pedidos, args=(cola_de_pedidos, clientes))
trabajador_2 = threading.Thread(target=atender_pedidos, args=(cola_de_pedidos, clientes))
trabajador_3 = threading.Thread(target=atender_pedidos, args=(cola_de_pedidos, clientes))


trabajador_1.start()
trabajador_2.start()
trabajador_3.start()


trabajador_1.join()
trabajador_2.join()
trabajador_3.join()

for cliente in clientes:
    print(f"La bandeja de {cliente} tiene: {clientes[cliente]}")

## Problema 5

Ahora se te presenta un videojuego que consiste de cuatro niveles distintos. Pero como en todo buen videojuego, solo puedes acceder a un nivel si completaste el anterior. Es por esto, que los antiguos desarrolladores dejaron el objeto `Event`: `evento_nivel_terminado`, que reciben como argumento todos los niveles.

Completa el código utilizando el objeto `Event` mencionado, pero **solo puedes agregar código dentro de la sección superior de la función `jugar_nivel` y bajo la sección comentada**.

In [ ]:
def jugar_nivel(nivel, evento_nivel):
    ##### SOLO AGREGAR CÓDIGO EN ESTA FUNCIÓN
    
    print(f"¡Jugando nivel {nivel}!")
    time.sleep(1)
    print(f"Batallando en el nivel {nivel}")
    time.sleep(3)
    print(f"Terminando el nivel {nivel}")
    
    
evento_nivel_terminado = threading.Event()

n1 = threading.Thread(target=jugar_nivel, args=[1, evento_nivel_terminado])
n2 = threading.Thread(target=jugar_nivel, args=[2, evento_nivel_terminado])
n3 = threading.Thread(target=jugar_nivel, args=[3, evento_nivel_terminado])
n4 = threading.Thread(target=jugar_nivel, args=[4, evento_nivel_terminado])

# =========== SOLO AGREGAR CÓDIGO DESDE AQUÍ HACIA ABAJO =============

n1.start()

n2.start()

n3.start()

n4.start()

## Problema 6

Juan ha decidio mostrar sus nuevas habilidades como programador creando un sistema que simula como dormiría una siesta un día cualquiera. Para ello implemento todo un sistema con *threads* y eventos para no pasarse de la hora.

Lamentablemente a Juan se le olvidó programar es sistema que activara su `alarma` luego de de 5 horas de siesta *(A Juan le gusta dormir mucho 😅)*. 

Es por esto que tú debes completar su programa y lograr que active la alarma **luego de 5 horas**, pero únicamente utilizando `Timer` de la librería `threading` y solo agregando código desde la sección comentada hacia abajo.


**Aclaración:** Interpreta una hora como un segundo en el programa, no te quedes esperando cinco horas a ver si funciona, por favor.

In [ ]:
def dormir(hora_actual, alarma):
    print(f"Tomaré una siesta, son las {hora_actual}")
    for i in range(15):
        if alarma.is_set():
            print("¡Gracias despertador, desperté a la hora!")
            return
        hora_actual += 1
        print(f"Estoy durmiendo a las {hora_actual}")
        time.sleep(1)
    print(f"¡Oh no!\nMe quedé dormido, son las {hora_actual}!!\n¡¡¡MALDITO DESPERTADOR!!!")
    
hora = 9
alarma = threading.Event()

thread = threading.Thread(target=dormir, args=[hora, alarma])
thread.start()

# ================ AGREGAR CÓDIGO DESDE AQUÍ ===================



## Problema 7

Ha llegado el Circo y tienen a sus tres mejores vendedores vendiendo entradas para la siguiente función. Para esto el jefe ha diseñado un código que simula el sistema de ventas, donde: 


- `asientos_vendidos` es una lista que contiene a los asientos, donde cada uno es un `bool` para indicar si está ocupado (`True`) o no (`False`).


- `entradas_vendidas` es el total de entradas vendidas por los vendedores.

Luego de un tiempo el jefe se da cuenta que hay un error en su simulación, pues nunca se logran vender correctamente el total de tickets. Debes analizar el siguiente código y utilizando `Lock` arregla el comportamiento del programa dónde se logran vender todos los tickets del circo. Intenta lograrlo utilizando un atributo de la clase `Circo`.

In [ ]:
from threading import Thread, Lock

class Circo:
    
    
    
    def __init__(self):
        self.entradas_vendidas = 0
        self.asientos_vendidos = [False for _ in range(1000000)]
        self.v1 = Thread(target=self.vendedor_1)
        self.v2 = Thread(target=self.vendedor_2)
        self.v3 = Thread(target=self.vendedor_3)
        
    def vender(self):
        self.v1.start()
        self.v2.start()
        self.v3.start()
        self.v1.join()
        self.v2.join()
        self.v3.join()

    def vendedor_1(self):
        for posicion in range(len(self.asientos_vendidos)):
            if not self.asientos_vendidos[posicion]:
                self.asientos_vendidos[posicion] = True
                self.entradas_vendidas += 1

    def vendedor_2(self):
        for posicion in range(len(self.asientos_vendidos)):
            if not self.asientos_vendidos[posicion]:
                self.asientos_vendidos[posicion] = True
                self.entradas_vendidas += 1

    def vendedor_3(self):
        for posicion in range(len(self.asientos_vendidos)):
            if not self.asientos_vendidos[posicion]:
                self.asientos_vendidos[posicion] = True
                self.entradas_vendidas += 1

                
cirque_du_soleil = Circo()
cirque_du_soleil.vender()

print(cirque_du_soleil.entradas_vendidas)

## Problema 8

En base al siguiente código, escribe el `output` que esperas se imprima al ejecutarlo. Luego responde las preguntas de al final.

In [ ]:
from random import shuffle

bebestibles = ["Vino"] * 15 + ["Pipeño"]
helados = ["Vainilla"] * 20 + ["Piña"]
shuffle(bebestibles)
shuffle(helados)

pipeño_encontrado = threading.Event()
helado_encontrado = threading.Event()

def busca_pipeño():
    print("¡Voy por el pipeño!")
    for bebestible in bebestibles:
        time.sleep(1)
        if bebestible == "Pipeño":
            print("¡Encontré el pipeño!")
            helado_encontrado.wait()
            pipeño_encontrado.set()
            print("¡Salud!")
            return
    
def busca_helado_de_piña():
    print("¡Voy por el helado de piña!")
    for helado in helados:
        time.sleep(1)
        if helado == "Piña":
            print("¡Encontré el helado!")
            pipeño_encontrado.wait()
            helado_encontrado.set()
            print("¡Salud!")
            return



thread_1 = threading.Thread(target=busca_pipeño)
thread_2 = threading.Thread(target=busca_helado_de_piña)

thread_1.start()
thread_2.start()
thread_1.join()
thread_2.join()
print("¡Ti-ki-ti-ki-ti!")

**¿Qué pasa que el código no termina de correr? ¿Puedes arreglarlo?**